## Types

In Julia everything has a type, and types are hierarchical. For instance, consider the datatype `Number`:

![](https://upload.wikimedia.org/wikipedia/commons/4/40/Type-hierarchy-for-julia-numbers.png)

In [ ]:
super(Number)

In [ ]:
subtypes(Number)

In [ ]:
subtypes(Real)

In [ ]:
subtypes(AbstractFloat)

In [ ]:
level = 0
function showtypetree(subtype)
    global level
    subtypelist = filter(asubtype -> asubtype != Any, subtypes(subtype))
    if length(subtypelist) > 0 
         println("\t" ^ level, subtype)        
         level += 1
         map(showtypetree, subtypelist)
         level -= 1
    else
         println("\t" ^ level, subtype)
    end    
end

In [ ]:
showtypetree(Number)

## Creating a `type`

In [ ]:
abstract Feline

type Jaguar <: Feline end
type Lion <: Feline end

In [ ]:
subtypes(Feline)

In [ ]:
showtypetree(Feline)

In [ ]:
abstract MyAbstractType

type MyType <: MyAbstractType
   foo
   bar::Int
end

In [ ]:
x = MyType("Hello World!", 10)

In [ ]:
x.foo

In [ ]:
x.bar

In [ ]:
x.foo = 3.0

In [ ]:
x.foo

### Ex: British Currency

The important task is to create a **constructor function**. This has the same name as the type, and accepts three values as arguments. The special `new()` function creates a new object with the passed-in values. Remember we're still inside the type definition — this is an inner constructor.

In [ ]:
type LSD
   pounds::Int 
   shillings::Int
   pence::Int
   
   function LSD(a,b,c)
    if a < 0 || b < 0 
      error("no negative numbers")
    end
    if c > 12 || b > 20
      error("too many pence or shillings")
    end
    new(a, b, c) 
   end   
end

In [ ]:
price1 = LSD(5,10,6)

In [ ]:
price2 = LSD(1,6,8)

In [ ]:
fieldnames(price1)

The next task is to make this new type behave in the same way as other Julia objects. For example, we can't add two prices:

In [ ]:
price1 + price2

and the output could definitely be improved:

In [ ]:
price2

In [ ]:
function +(a::LSD, b::LSD)
  newpence = a.pence + b.pence
  newshillings = a.shillings + b.shillings
  newpounds = a.pounds + b.pounds
  subtotal = newpence + newshillings * 12 + newpounds * 240
  (pounds, balance) = divrem(subtotal, 240)
  (shillings, pence) = divrem(balance, 12)
  LSD(pounds, shillings, pence)
end

In [ ]:
price1 + price2

The next problem to address is the unattractive presentation of LSD objects. This is fixed in exactly the same way, by adding a new method, but this time to the `show()` function, which belongs to the Base environment:

In [ ]:
function Base.show(io::IO, money::LSD)
    print(io, "£$(money.pounds).$(money.shillings)s.$(money.pence)d")
end

In [ ]:
price1 + price2

In [ ]:
typealias Price LSD

In [ ]:
Price(1, 19, 11)

In [ ]:
include("types.jl")

In [ ]:
x = linspace(1., 100., 1000)
data = RadialVelocityData(x, x, x);

In [ ]:
global data = RadialVelocityData(x, x, x)

In [ ]:
t = Array{Float64, 1}

## Modules  

We will now import the Sundial module. See the file Sundial.jl.

In [ ]:
push!(LOAD_PATH, ".")

In [ ]:
using Sundial

The value of `latitude` is available, and can be used without the prefix, because the module exported it and because we loaded the module with `using`

In [ ]:
latitude

In [ ]:
Sundial.latitude

In [ ]:
Sundial.latitude = 2

In [ ]:
Sundial.get_lat()

In [ ]:
latitude = 40

In [ ]:
latitude

In [ ]:
Sundial.latitude

In [ ]:
Main.latitude

In [ ]:
Sundial.get_lat()

In [ ]:
Sundial.set_lat(45)

In [ ]:
Sundial.get_lat()

The `set_lat()` function used the `global` keyword at the start of the definition. This means that code inside the function definition can change the value.

In [ ]:
Sundial.latitude

In [ ]:
using RV

In [ ]:
if RV.time == nothing
    println("You must set RV time grid using RV.set_time()")
end 
if RV.rv == nothing
    println("You must set RV data using RV.set_rv()")
end 
if RV.err == nothing
    println("You must set RV error using RV.set_err()")
end 

In [ ]:
RV.set_time(collect(linspace(1., 100., 100)));

In [ ]:
err = 10.0 .* randn(100)
RV.set_rv((100.0.*cos(RV.time)).+ 2.0.*err);

In [ ]:
RV.set_err(err);

In [ ]:
time_truth = collect(linspace(1., 100., 1000));
rv_truth = 100.0.*cos(time_truth);

In [ ]:
using PyPlot

In [ ]:
errorbar(RV.time, RV.rv, yerr=RV.err, ms=5, fmt="o", c="k")
plot(time_truth, rv_truth, c="blue", lw=0.5, ls="-")
xlabel("Time [Days]")
ylabel("RV [m/s]")